<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=6e3d131480ee0ecfd2defc4b805055f6ace2878e4d5866fe949771a443c7a7bd
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.4
    Uninstalling protobuf-6.33.4:
      Successfully uninstalled protobuf-6.33.4


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 1.0
    Uninstalling yfinance-1.0:
      Successfully uninstalled yfinance-1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-01-21 09:31:43
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 14
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  1.68 K
Current:  1.35 C
-------------------
Today PnL: -55.92 K (-0.41%)
Current PnL: -33.73 L (-21.51%)
CY Booked + Current PnL: -19.44 L (-12.4%)
-------------------
Total profit:  1.21 L
Total loss:  -34.94 L
-------------------
Total Booked + Current PnL: 7.54 L (5.81%)
Total Booked PnL: 41.27 L (31.79%)
Curr Year Booked PnL: 14.29 L (10.56%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.34 C
Est FTT PnL: 99.08 L (73.2%)
Deployed:  1.30 C
Current:  1.35 C
CAGR/XIRR %: 2.21%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,NESTLEIND,1377.00,0.26,14.53,6.00,21.40,18239.0,38563.0,303989.0,6.33,54.0,X-LC,7.07,14.0,2.11,2.25,21.78,XY25,NTT,FMCG
77,TTKPRESTIG,770.00,0.14,-23.21,30.34,0.09,23480.0,-23389.0,77388.0,79.72,37.0,M-SC,4.96,253.0,-1.00,0.57,1.82,OX40N,NTT,DURABLES
17,COALINDIA,484.83,0.22,2.35,16.49,19.23,25394.0,3534.0,153994.0,19.88,53.0,L-LC,10.53,182.0,0.14,1.14,24.12,XY25,ATH,MINING
49,MASFIN,398.61,-0.38,-7.38,31.77,22.05,28831.0,-7230.0,90750.0,-20.05,39.0,H-SC,4.73,168.0,-0.25,0.67,31.91,XR,ATH,FINANCE
39,INDIGOPNTS,1408.00,-1.54,-20.29,25.48,0.02,35453.0,-35419.0,139140.0,112.05,38.0,M-SC,8.80,240.0,-1.00,1.03,21.03,OX40N,NTT,PAINTS


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,3.08,-5.13,117.90,106.72,137952.0,-6328.0,117008.0,-53.11,50.0,H-SC,8.72,138.0,-0.05,0.86,17.05,AR,ATH,MISC
28,GREENPANEL,537.00,2.42,-42.49,148.88,43.14,156765.0,-77782.0,105296.0,135.79,36.0,M-SC,15.09,231.0,-0.50,0.78,2.42,XY24,NTT,MISC
0,5PAISA,593.00,1.45,-30.85,63.27,12.90,95595.0,-67405.0,151091.0,122.36,53.0,H-SC,7.40,173.0,-0.71,1.12,25.63,OX40N,NTT,FINANCE
4,ALKYLAMINE,4546.37,1.32,-33.26,190.11,93.63,128106.0,-33578.0,67385.0,-28.56,45.0,H-SC,16.58,143.0,-0.26,0.50,1.87,SR,ATH,CHEMICALS
10,BANDHANBNK,400.00,1.16,-32.17,183.69,92.42,347064.0,-89619.0,188940.0,32.90,39.0,H-SC,10.66,169.0,-0.26,1.40,9.19,XY24,NTT,BANKS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,JCHAC,2282.00,-4.21,-41.01,69.49,-0.01,52394.0,-52407.0,75398.0,15334.78,29.0,M-SC,14.06,235.0,-1.00,0.56,0.00,OX40N,BTT,AC
55,RAJESHEXPO,518.00,-2.99,-69.38,226.81,0.07,95265.0,-95175.0,42002.0,1439.14,33.0,L-SC,6.26,269.0,-1.00,0.31,4.46,OX40N,NTT,JEWELLERY
31,HAVELLS,2069.16,-2.85,-16.26,58.20,32.48,159852.0,-53332.0,274659.0,-20.44,27.0,X-MC,5.29,69.0,-0.33,2.03,0.00,X40,ATH,ELECTRICAL
56,RAJOOENG,143.10,-2.60,-33.81,110.94,39.61,75262.0,-34660.0,67840.0,-55.26,50.0,H-SC,28.71,136.0,-0.46,0.50,9.81,AR,ATH,MISC
78,UNITDSPR,1644.00,-2.36,-5.42,27.69,20.77,61320.0,-12694.0,221450.0,-13.65,28.0,X-MC,2.55,70.0,-0.21,1.64,1.12,X40N,NTT,BREWERIES


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.87,-8.81,127.99,107.91,172326.0,-13004.0,134640.0,-25.82,34.0,M-SC,10.00,220.0,-0.08,0.99,0.39,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-1.54,-20.29,25.48,0.02,35453.0,-35419.0,139140.0,112.05,38.0,M-SC,8.80,240.0,-1.00,1.03,21.03,OX40N,NTT,PAINTS
72,TATAELXSI,9161.00,-0.46,-22.72,70.85,32.03,72181.0,-29957.0,101878.0,-16.37,47.0,H-SC,6.43,157.0,-0.42,0.75,14.11,OX40N,NTT,IT
77,TTKPRESTIG,770.00,0.14,-23.21,30.34,0.09,23480.0,-23389.0,77388.0,79.72,37.0,M-SC,4.96,253.0,-1.00,0.57,1.82,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-1.30,-25.28,51.86,13.47,104498.0,-68166.0,201501.0,-69.28,38.0,H-SC,4.39,159.0,-0.65,1.49,2.46,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
17,COALINDIA,484.83,0.22,2.35,16.49,19.23,25394.0,3534.0,153994.0,19.88,53.0,L-LC,10.53,182.0,0.14,1.14,24.12,XY25,ATH,MINING


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,0.15,-1.46,74.82,72.27,124930.0,-2474.0,166974.0,-15.61,32.0,M-LC,4.54,101.0,-0.02,1.23,4.62,XR,NTT,IT
38,INDIAMART,4810.62,3.08,-5.13,117.90,106.72,137952.0,-6328.0,117008.0,-53.11,50.0,H-SC,8.72,138.0,-0.05,0.86,17.05,AR,ATH,MISC
25,FINCABLES,1641.55,-0.87,-8.81,127.99,107.91,172326.0,-13004.0,134640.0,-25.82,34.0,M-SC,10.00,220.0,-0.08,0.99,0.39,OX40N,ATH,CABLES
85,ZYDUSLIFE,1286.17,0.35,-7.57,46.74,35.63,90538.0,-15868.0,193706.0,-20.73,36.0,H-MC,4.72,119.0,-0.18,1.43,7.84,AR,ATH,PHARMA
37,IEX,219.00,1.02,-10.68,66.39,48.63,118578.0,-21346.0,178608.0,-40.51,40.0,H-SC,13.48,137.0,-0.18,1.32,1.02,XR,NTT,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,AWL,485.00,-0.99,-32.60,133.22,57.19,292836.0,-106313.0,219814.0,-68.10,15.0,X-MC,12.95,58.0,-0.36,1.62,0.00,XY24,NTT,FMCG
58,RELAXO,1176.00,-0.21,-52.84,222.99,52.31,152639.0,-76709.0,68451.0,-49.76,18.0,X-SC,9.14,92.0,-0.50,0.51,0.00,X40N,NTT,FOOTWEAR
43,ITC,452.00,0.51,-19.62,37.83,10.78,73074.0,-47155.0,193163.0,-52.19,19.0,X-LC,12.76,1.0,-0.65,1.43,0.51,X40,NTT,FMCG
11,BATAINDIA,2096.00,-0.88,-43.21,138.07,35.20,100892.0,-55597.0,73073.0,-4.26,19.0,X-SC,18.18,93.0,-0.55,0.54,0.00,X40,NTT,FOOTWEAR
53,PGHH,17907.65,-0.17,-10.38,49.73,34.19,95163.0,-22160.0,191360.0,-36.94,21.0,X-MC,5.44,60.0,-0.23,1.41,0.00,X40,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4311.59,-0.35,-15.00,39.47,18.55,126898.0,-56731.0,321506.0,-26.86,42.0,X-LC,0.97,3.0,-0.45,2.38,9.38,X40,ATH,IT
35,ICICIGI,2252.93,-0.67,-2.04,21.83,19.34,42792.0,-4083.0,196026.0,-22.62,33.0,X-MC,2.18,71.0,-0.10,1.45,13.57,X40,ATH,INSURANCE
20,DABUR,735.00,-0.06,-1.31,45.60,43.70,111642.0,-3240.0,244828.0,-9.42,44.0,X-MC,2.25,73.0,-0.03,1.81,13.55,XY24,BTT,FMCG
33,HINDUNILVR,2922.00,0.42,-7.06,22.30,13.67,53279.0,-18143.0,238920.0,-13.94,55.0,X-LC,2.26,9.0,-0.34,1.77,12.79,XY25,NTT,FMCG
45,JIOFIN,387.00,0.43,-14.23,45.13,24.48,103492.0,-38046.0,229319.0,-15.63,27.0,X-LC,2.45,37.0,-0.37,1.69,32.92,XY24,BTT,FINANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.00,-0.34,-28.93,130.99,64.15,221504.0,-68851.0,169100.0,-58.50,36.0,X-SC,6.71,84.0,-0.31,1.25,0.0,XY24,BTT,CEMENT
7,AWL,485.00,-0.99,-32.60,133.22,57.19,292836.0,-106313.0,219814.0,-68.10,15.0,X-MC,12.95,58.0,-0.36,1.62,0.0,XY24,NTT,FMCG
11,BATAINDIA,2096.00,-0.88,-43.21,138.07,35.20,100892.0,-55597.0,73073.0,-4.26,19.0,X-SC,18.18,93.0,-0.55,0.54,0.0,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-0.93,-26.62,78.89,31.26,133583.0,-61440.0,169328.0,-65.70,25.0,X-MC,23.98,56.0,-0.46,1.25,0.0,X40N,ATH,IT
31,HAVELLS,2069.16,-2.85,-16.26,58.20,32.48,159852.0,-53332.0,274659.0,-20.44,27.0,X-MC,5.29,69.0,-0.33,2.03,0.0,X40,ATH,ELECTRICAL


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,424.00,-0.11,-33.10,113.53,42.84,49371.0,-21519.0,43487.0,-56.47,33.0,X-SC,23.00,83.0,-0.44,0.32,0.00,XY24,NTT,MISC
58,RELAXO,1176.00,-0.21,-52.84,222.99,52.31,152639.0,-76709.0,68451.0,-49.76,18.0,X-SC,9.14,92.0,-0.50,0.51,0.00,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.88,-43.21,138.07,35.20,100892.0,-55597.0,73073.0,-4.26,19.0,X-SC,18.18,93.0,-0.55,0.54,0.00,X40,NTT,FOOTWEAR
50,MEDANTA,1486.00,-1.81,-11.81,40.69,24.08,45555.0,-14993.0,111957.0,-18.97,28.0,X-SC,6.90,91.0,-0.33,0.83,3.82,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,0.15,-17.84,74.44,43.32,99616.0,-29052.0,133820.0,-29.42,45.0,X-SC,8.18,90.0,-0.29,0.99,2.75,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,0.51,-19.62,37.83,10.78,73074.0,-47155.0,193163.0,-52.19,19.0,X-LC,12.76,1.0,-0.65,1.43,0.51,X40,NTT,FMCG
75,TMPV,600.00,0.67,-30.29,76.42,22.98,120336.0,-68432.0,157466.0,-27.96,34.0,X-LC,3.61,2.0,-0.57,1.16,0.67,XY24,NTT,AUTO
73,TCS,4311.59,-0.35,-15.00,39.47,18.55,126898.0,-56731.0,321506.0,-26.86,42.0,X-LC,0.97,3.0,-0.45,2.38,9.38,X40,ATH,IT
81,VBL,671.64,-0.84,-2.03,38.48,35.67,119069.0,-6412.0,309430.0,-13.74,46.0,X-LC,2.93,5.0,-0.05,2.29,11.54,X40N,ATH,FMCG
41,INFY,1972.00,-0.88,11.94,19.93,34.24,69802.0,37348.0,350236.0,-12.78,54.0,X-LC,7.25,6.0,0.54,2.59,21.21,X40,NTT,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,1.13,-32.42,96.65,32.89,50427.0,-25032.0,52175.0,-744.47,57.0,L-MC,10.48,259.0,-0.50,0.39,43.69,XR,NTT,BANKS
17,COALINDIA,484.83,0.22,2.35,16.49,19.23,25394.0,3534.0,153994.0,19.88,53.0,L-LC,10.53,182.0,0.14,1.14,24.12,XY25,ATH,MINING
5,ASIANPAINT,3460.26,-0.24,-6.70,29.64,20.95,69621.0,-16878.0,234890.0,-7.66,35.0,X-LC,12.61,36.0,-0.24,1.74,26.54,X40,ATH,PAINTS
32,HCLTECH,1908.19,-0.65,8.79,13.83,23.83,36400.0,21271.0,263195.0,13.61,57.0,X-LC,6.32,13.0,0.58,1.94,26.27,X40,ATH,IT
51,NESTLEIND,1377.00,0.26,14.53,6.00,21.40,18239.0,38563.0,303989.0,6.33,54.0,X-LC,7.07,14.0,2.11,2.25,21.78,XY25,NTT,FMCG


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.00,1.45,-30.85,63.27,12.90,95595.0,-67405.0,151091.0,122.36,53.0,H-SC,7.40,173.0,-0.71,1.12,25.63,OX40N,NTT,FINANCE
17,COALINDIA,484.83,0.22,2.35,16.49,19.23,25394.0,3534.0,153994.0,19.88,53.0,L-LC,10.53,182.0,0.14,1.14,24.12,XY25,ATH,MINING
40,INDUSINDBK,1800.00,1.13,-32.42,96.65,32.89,50427.0,-25032.0,52175.0,-744.47,57.0,L-MC,10.48,259.0,-0.50,0.39,43.69,XR,NTT,BANKS
62,SATIN,274.00,0.59,-17.10,83.16,51.83,190468.0,-47246.0,229038.0,-25.10,57.0,H-SC,3.31,148.0,-0.25,1.69,11.58,XY24,NTT,FINANCE
56,RAJOOENG,143.10,-2.60,-33.81,110.94,39.61,75262.0,-34660.0,67840.0,-55.26,50.0,H-SC,28.71,136.0,-0.46,0.50,9.81,AR,ATH,MISC


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.58
1,25,44.82
2,50,76.79


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    42.50
MC    30.59
LC    26.95
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.27
X40      24.16
X40N     12.89
XY25     10.96
XR        8.77
AR        8.44
OX40N     7.59
SR        0.96
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.18
X-LC    23.53
H-SC    22.76
M-SC    11.06
X-SC     7.91
H-MC     4.57
M-MC     1.45
M-LC     1.23
L-LC     1.14
H-LC     1.05
L-SC     0.77
L-MC     0.39
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.53,-11.09,47.54
IT,12.83,-23.06,84.53
FINANCE,11.10,-18.55,68.31
MISC,7.11,-35.01,90.24
PAINTS,5.78,-18.14,35.69
ELECTRICAL,5.73,-21.17,65.54
INSURANCE,4.76,-5.35,41.40
PHARMA,3.97,-9.45,43.91
BANKS,2.81,-40.94,126.34


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3415865.0,21
AR,1428738.0,10
XR,1306108.0,12
X40,1243425.0,15
X40N,999421.0,9
OX40N,763892.0,10
XY25,459221.0,7
SR,291381.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3713715.0,24
X-MC,1760640.0,16
M-SC,1563481.0,15
X-LC,1031503.0,13
X-SC,850158.0,8
H-MC,461120.0,3
L-SC,186734.0,2
M-LC,124930.0,1
H-LC,91473.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1301575.0      6
           AR         951900.0      5
M-SC       XY24       886387.0      6
H-SC       XR         843436.0      7
X-MC       X40        600804.0      7
           XY24       473846.0      3
           X40N       444141.0      4
X-LC       X40        441754.0      6
M-SC       OX40N      343204.0      5
X-SC       X40N       332861.0      3
H-SC       OX40N      325423.0      4
X-SC       XY24       316430.0      3
H-SC       SR         291381.0      2
H-MC       AR         247321.0      2
X-MC       XY25       241849.0      2
X-LC       XY24       223828.0      2
           X40N       222419.0      2
H-MC       XY24       213799.0      1
X-SC       X40        200867.0      2
M-SC       XR         195846.0      2
X-LC       XY25       143502.0      3
M-SC       AR         138044.0      2
M-LC       XR         124930.0      1
L-SC       OX40N       95265.0      1
H-LC       AR          91473.0      1
L-SC       XR          91469.0      1
L-MC       XR          50427.0      1
M-MC       XY25        48476.0      1
L-LC       XY25        25394.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 26.0 seconds
